This is a notebook to examine the modeling to answer the problem of assiging a genre to a song based on Spotify
metadata of a song.

Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Read in clean data

In [ ]:
data = pd.read_csv('./data/clean_data.csv')